In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Model
from autoencoder.supervise import equation
import numpy as np
from fenics import *
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})

In [2]:
# load data
dataset = equation.Dataset('poisson')
dataset.load()
x_train, y_train = dataset.train
x_test, y_test = dataset.test
x_val, y_val = dataset.validate

In [3]:
# network parameters
latent_dim = 3
batch_size = 25
input_shape = (100,)
input_u = Input(shape=input_shape)

In [4]:
@tf.custom_gradient
def solver(theta):
    """
    Solves constant force 1D Poisson equation
    and returns the solution as a numpy array
    with length soln_dim.
    """
    # convert eager tensor to numpy array
    # with shape (batch_size, num_params)
    theta = theta.numpy()
    batch_size = theta.shape[0]
    
    # Create mesh and define function space
    mesh = UnitIntervalMesh(99)
    V = FunctionSpace(mesh, 'P', 1)

    # Define variational problem
    u = TrialFunction(V)
    v = TestFunction(V)
    a = grad(u)[0] * grad(v)[0] * dx
    
    batch_solns = np.zeros((batch_size, 100))
    for idx, params in enumerate(theta):
        u0, u1, c = params
        c = Constant(c)
        L = c * v * dx
        
        # Define boundary condition
        u_D = Expression('x[0] == 0 ? u0: u1',
                        u0 = u0,
                        u1 = u1,
                        degree = 2)
        def boundary(x, on_boundary):
            return on_boundary
        bc = DirichletBC(V, u_D, boundary)

        # Compute solution
        u = Function(V)
        solve(a == L, u, bc)
        vertex_values_u = u.compute_vertex_values(mesh)
        batch_solns[idx, :] = vertex_values_u
    
    def d_solver(dy):
        d = tf.eye(25,3)
        return d
    return batch_solns.astype('float32'), d_solver

In [5]:
class Fenics(Layer):
    
    def __init__(self, name='fenics'):
        super(Fenics, self).__init__()
        
    def call(self, inputs):
        return solver(inputs)

class SemanticAutoEncoder(Model):
    
    def __init__(self, name='sae'):
        super(SemanticAutoEncoder, self).__init__()
        self.dense1 = Dense(50, 'relu', name='hidden_1')
        self.dense2 = Dense(20, 'relu', name='hidden_2')
        self.latent_dense = Dense(3, 'linear', trainable=False, name='theta')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        theta = self.latent_dense(x)
        u_reconstructed = Fenics()(theta)
        return u_reconstructed 

In [6]:
sae = SemanticAutoEncoder()
optim = tf.keras.optimizers.Adam(epsilon=1e-12)
sae.compile(optim, 'mse')
sae.run_eagerly = True
x_train = x_train.astype('float32')
sae.fit(x_train, x_train, epochs=30, batch_size=batch_size, verbose=2)

Epoch 1/30
48/48 - 3s - loss: 350.9905
Epoch 2/30
48/48 - 3s - loss: 7437.0386
Epoch 3/30
48/48 - 4s - loss: 55238.2891
Epoch 4/30
48/48 - 3s - loss: 261254.7500
Epoch 5/30
48/48 - 3s - loss: 968080.5625
Epoch 6/30
48/48 - 3s - loss: 2649529.5000
Epoch 7/30
48/48 - 3s - loss: 5607879.5000
Epoch 8/30
48/48 - 3s - loss: 11968852.0000
Epoch 9/30
48/48 - 4s - loss: 22878442.0000
Epoch 10/30


KeyboardInterrupt: 

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
input_u = Input(shape=input_shape)
x = Dense(50, 'relu', name='hidden_1')(input_u)
x = Dense(20, 'relu', name='hidden_2')(x)
x = Dense(3, 'linear', name='theta')(x)
fenics = Decoder()
fenics.run_eagerly = True
u_hat = fenics(x)
   
model = Model(input_u, u_hat)

model.compile('adam', loss='mse')
model.run_eagerly = True
model.fit(x_train, x_train, epochs=10, batch_size=25)